In [122]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

import sklearn
from sklearn.linear_model import LogisticRegression

In [78]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.layers(x)

In [65]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=5000, n_features=10, n_informative=2, n_redundant=2, n_repeated=0, n_classes=3, n_clusters_per_class=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [84]:
from torch.utils.data import Dataset

class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    

# init dataset
train_ds = MyCustomDataset(X_train, y_train)
test_ds = MyCustomDataset(X_test, y_test)

In [85]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=16)

## model fitting

In [140]:
model = SimpleMLP(10,30,3)
loss_fn = nn.CrossEntropyLoss()  # For multiclass classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()   # Clear previous gradients
        loss.backward()         # Compute gradients
        optimizer.step()        # Update parameters

        if batch_idx % 50 == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}")

Epoch 0, Batch 0, Loss: 1.0954
Epoch 0, Batch 50, Loss: 0.9931
Epoch 0, Batch 100, Loss: 0.7678
Epoch 0, Batch 150, Loss: 0.7263
Epoch 0, Batch 200, Loss: 0.6568
Epoch 1, Batch 0, Loss: 0.6369
Epoch 1, Batch 50, Loss: 0.7054
Epoch 1, Batch 100, Loss: 0.7733
Epoch 1, Batch 150, Loss: 0.6414
Epoch 1, Batch 200, Loss: 0.7035
Epoch 2, Batch 0, Loss: 0.6402
Epoch 2, Batch 50, Loss: 0.6571
Epoch 2, Batch 100, Loss: 0.6186
Epoch 2, Batch 150, Loss: 0.5615
Epoch 2, Batch 200, Loss: 0.7197
Epoch 3, Batch 0, Loss: 0.6672
Epoch 3, Batch 50, Loss: 0.6926
Epoch 3, Batch 100, Loss: 0.5678
Epoch 3, Batch 150, Loss: 0.6364
Epoch 3, Batch 200, Loss: 0.6776
Epoch 4, Batch 0, Loss: 0.7048
Epoch 4, Batch 50, Loss: 0.7150
Epoch 4, Batch 100, Loss: 0.6199
Epoch 4, Batch 150, Loss: 0.5608
Epoch 4, Batch 200, Loss: 0.6402
Epoch 5, Batch 0, Loss: 0.7201
Epoch 5, Batch 50, Loss: 0.5573
Epoch 5, Batch 100, Loss: 0.6736
Epoch 5, Batch 150, Loss: 0.6474
Epoch 5, Batch 200, Loss: 0.5566
Epoch 6, Batch 0, Loss: 0.55

## prediction on test set

In [141]:
# prediction on test set
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = model(inputs)
        # For classification, you may want the predicted class:
        preds = outputs.argmax(axis=1)  # For binary classification
        # For multi-class: preds = torch.argmax(outputs, dim=1)
        all_preds += preds.numpy().tolist()
        all_targets += targets.numpy().tolist()

In [144]:
sklearn.metrics.multilabel_confusion_matrix(all_targets, all_preds)
f1 = sklearn.metrics.f1_score(all_targets, all_preds, average="micro")
acc = sklearn.metrics.accuracy_score(all_targets, all_preds)

print(f1, acc)

0.9163636363636365 0.9163636363636364


## Learn a basic linear model as benchmark

In [126]:
mod = LogisticRegression()
mod = mod.fit(X_train, y_train)
logreg_preds = mod.predict(X_test) 

In [145]:
sklearn.metrics.multilabel_confusion_matrix(y_test, logreg_preds)
f1 = sklearn.metrics.f1_score(y_test, logreg_preds, average="micro")
acc = sklearn.metrics.accuracy_score(y_test, logreg_preds)
print(f1, acc)

0.8896969696969697 0.8896969696969697
